In [1]:
import pandas as pd # data manipulation
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np #  mathematical support for large, multi-dimensional arrays and matrices
import os

In [2]:
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import collections

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-nmym4uan because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import re

### Set Directory

In [4]:
os.getcwd()

'/scratch/interns202010/jmakins/jupyter-dir'

In [99]:
try:
    os.chdir('/group/interns202010/jmakins/Data')
except:
    pass

In [100]:
os.listdir()

['world_soccer_database.sqlite',
 'events',
 'referees.json',
 'players_old.json',
 'players.json',
 'competitions.json',
 'teams.json',
 'coaches.json',
 'players_old.csv',
 'matches',
 '.DS_Store']

In [101]:
Teams = pd.read_json('teams.json')
Competitions = pd.read_json('competitions.json')
Players = pd.read_json('players.json')
Coaches = pd.read_json('coaches.json') # Managers

In [102]:
#Players_old = pd.read_json('players_old.json')

In [103]:
#Competitions`['CompId'] = Competitions['wyId']

In [104]:
#Competitions.drop(['wyId', 'CountryId'], inplace = True, axis = 1)

In [105]:
try:
    os.chdir('/group/interns202010/jmakins/Data/matches')
except:
    pass
England = pd.read_json('matches_England.json')
France = pd.read_json('matches_France.json')
Italy = pd.read_json('matches_Italy.json')
Spain = pd.read_json('matches_Spain.json')
Germany = pd.read_json('matches_Germany.json')
#World_Cup = pd.read_json('matches_World_Cup.json')
#Euro_Champs = pd.read_json('matches_European_Championship.json')

In [106]:
# Adding country labels to match observations
England['Country'] = 'England'
France['Country'] = 'France'
Italy['Country'] = 'Italy'
Spain['Country'] = 'Spain'
Germany['Country'] = "Germany"
#World_Cup['Country'] = 'World Cup'
#Euro_Champs['Country'] = 'European Champs'

In [107]:
try:
    os.chdir('/group/interns202010/jmakins/Data/events')
except:
    pass

In [108]:
os.listdir()

['events_Spain.json',
 'events_Germany.json',
 'events_France.json',
 'events_European_Championship.json',
 'events_England.json',
 'events_World_Cup.json',
 '.DS_Store',
 'events_Italy.json']

In [109]:
#test = pd.read_json('events_England.json')

In [110]:
#Matches.loc[((Matches['homeTeamId']==1644) & (Matches['awayTeamId']==1659)) | ((Matches['homeTeamId']==1659) & (Matches['awayTeamId']==1644))]


In [111]:
#Events_England[Events_England['matchId']==2500037]

In [112]:
#Events_England[(Events_England['subEventDescription'].str.match('goal ')) & (Events_England['Result']=="Failure") & (Events_England['subEventName']=="Shot")]


In [113]:
#Events_England.loc[(Events_England['subEventDescription'].str.match("goal ")) & (Events_England['Result']=="Success")].subEventName.unique()


In [114]:
Events_France = pd.read_json('events_France.json')
Events_Spain = pd.read_json('events_Spain.json')
Events_Germany = pd.read_json('events_Germany.json')
#Events_EuroChamps = pd.read_json('events_European_Championship.json')
#Events_World_Cup = pd.read_json('events_World_Cup.json')
Events_Italy = pd.read_json('events_Italy.json')
Events_England = pd.read_json('events_England.json')

In [115]:
## We have a function for this now further down to be applied across all european regions

## Solution to convert second half time into 
# secs_to_add = list(Events_England[Events_England['matchPeriod']=="1H"].groupby('matchId').tail(1)['eventSec'])
# match_ids = list(Events_England["matchId"].unique())

# for idx in range(0, len(match_ids)):
#     Events_England['eventSec'] = list(np.where(
#        (Events_England['matchId'] == match_ids[idx]) & (Events_England['matchPeriod'] == "2H") , Events_England['eventSec'] + secs_to_add[idx], Events_England['eventSec']
#        ))

In [116]:
# Example of an events table
Events_England[Events_England['matchId'] == 2499719].tail()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
1763,8,Launch,[{'id': 1802}],8480,"[{'y': 53, 'x': 7}, {'y': 49, 'x': 46}]",2499719,Pass,1631,2H,2992.491575,84,177961040
1764,8,Head pass,[{'id': 1801}],49876,"[{'y': 51, 'x': 54}, {'y': 58, 'x': 73}]",2499719,Pass,1609,2H,2994.900590,82,177961035
1765,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1216}, {'id'...",7870,"[{'y': 58, 'x': 73}, {'y': 0, 'x': 0}]",2499719,Shot,1609,2H,2997.086392,100,177961036
1766,5,Ball out of the field,[],0,"[{'y': 48, 'x': 3}, {'y': 100, 'x': 100}]",2499719,Interruption,1631,2H,2998.963701,50,177961041
1767,3,Goal kick,[],8480,"[{'y': 100, 'x': 100}, {'y': 100, 'x': 100}]",2499719,Free Kick,1631,2H,3006.647279,34,177961042


### Data Cleaning

In [117]:
Matches = pd.concat([England, France, Italy, Spain, Germany], ignore_index=True)

In [118]:
Matches.drop(['duration'], inplace=True, axis =1) # removing groupName variable from World Cups

In [119]:
Players.drop(["passportArea"], inplace = True, axis = 1)

In [120]:
Players = Players.rename(columns ={'wyId': 'playerId'}) # rename for convenience

In [121]:
Teams = Teams.rename(columns = {'wyId': 'teamId'})
Players = Players.rename(columns = {'currentTeamId': 'teamId'})
Players = pd.merge(Players,Teams[['teamId', 'officialName']], on = 'teamId').rename(columns = {'officialName': 'clubName'})

In [122]:
Players['Position'] = Players.role.apply(pd.Series)['code3'].values

In [123]:
Players['birthCountry'] = Players.birthArea.apply(pd.Series)['name'].values

In [124]:
Players.drop(["birthArea", 'role'], inplace = True, axis = 1)

### Manipulate Match Data

In [125]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match

In [126]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for x in teams:
    homeTeam.append(x[0])
    awayTeam.append(x[1])

In [127]:
# add columns for additional match statistics
Matches["homeManagerId"] = None
for name in ["Score", "ScoreHT", "ScoreET", "ScoreP"]:
    colname = "home" + name
    colname2 = "away" + name
    Matches[colname] = None
    Matches[colname2] = None

In [128]:
# parse Home and Away teams as features from nest dictionary
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for game, team in enumerate(teams):
    if Matches.teamsData[game:game+1][game][team[0]]['side'] == 'home':
        homeTeam.append(team[0])
        awayTeam.append(team[1])
    else:
        awayTeam.append(team[0])
        homeTeam.append(team[1])

In [129]:
def get_team_stats(df, team): # function to parse team match stats from dictionary embedded in column teamsData from the Matches dataframe
    
    Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId = [],[],[],[],[],[],[],[]
    
    for game in range(0, len(df)):

        Bench.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['bench'])['playerId']))
        Starters.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['lineup'])['playerId']))
        Manager.append(df.teamsData[game][team[game]]['coachId'])
        Score.append(df.teamsData[game][team[game]]['score'])
        ScoreHT.append(df.teamsData[game][team[game]]['scoreHT'])
        ScoreET.append(df.teamsData[game][team[game]]['scoreET'])
        ScoreP.append(df.teamsData[game][team[game]]['scoreP'])
        teamId.append(df.teamsData[game][team[game]]['teamId'])
    
    return Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId

homeBench, homeStarters, homeManager, homeScore, homeScoreHT, homeScoreET, homeScoreP, homeTeamId = get_team_stats(Matches, homeTeam)
awayBench, awayStarters, awayManager, awayScore, awayScoreHT, awayScoreET, awayScoreP, awayTeamId = get_team_stats(Matches, awayTeam)


In [130]:
# Automating Bench Line Up Column Generation

homeColNames = []
for num in range(1,14): # add columns for home bench playerId
     homeColNames.append("homeBenchPlayer" + str(num))
        
awayColNames = []
for num in range(1,14): # add columns for away bench playerId
    awayColNames.append("awayBenchPlayer" + str(num))
    


In [131]:
# Inspecting bench counts as during errror handling reveals some matches had bench sizes of between 8 and 13 for certain fixtures
import collections
collections.Counter(list(map(lambda x: len(x), homeBench)))

Counter({7: 1425, 6: 18, 5: 1, 8: 5, 12: 205, 10: 49, 11: 101, 9: 21, 13: 1})

In [132]:
# Adding Bench Line Ups to DataFrame

homeBench2 = pd.DataFrame(homeBench,
     columns=homeColNames)
awayBench2 = pd.DataFrame(awayBench,
     columns=awayColNames)


In [133]:
# Automating Starting Line Up Column Generation

homeColNames =[]
for num in range(1,12): # add columns for starting playerId
     homeColNames.append("homePlayer" + str(num))
        
awayColNames = []
for num in range(1,12): # add columns for starting playerId
    awayColNames.append("awayPlayer" + str(num))


In [134]:
# Joining starting line ups, benches to matches dataframe

homeStarters2 = pd.DataFrame(homeStarters,
     columns=homeColNames)
awayStarters2 = pd.DataFrame(awayStarters,
     columns=awayColNames)

Matches = Matches.join([homeStarters2, awayStarters2, homeBench2, awayBench2])


In [135]:
# add individual columns of match data
Matches['homeManager'] = homeManager
Matches['homeScore'] = homeScore
Matches['homeScoreHT'] = homeScoreHT
Matches['homeScoreET'] = homeScoreET
Matches['homeScoreP'] = homeScoreP
Matches['homeTeamId'] = homeTeamId
Matches['awayManager'] = awayManager
Matches['awayScore'] = awayScore
Matches['awayScoreHT'] = awayScoreHT
Matches['awayScoreET'] = awayScoreET
Matches['awayScoreP'] = awayScoreP
Matches['awayTeamId'] = awayTeamId

In [136]:
Matches.drop(["teamsData"], inplace = True, axis = 1)

In [137]:
# Classify Match Result as W, L or D for teams
conditions = [
    (Matches['winner'] == 0),
    (Matches['winner'] == Matches['awayTeamId']),
    (Matches['winner'] == Matches['homeTeamId'])
    ]

# create a list of the values we want to assign for each condition
values = [0, -1, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
Matches['Result'] = np.select(conditions, values)

###### Relating table to access Referee Data

In [138]:
# os.chdir('/group/interns202010/jmakins/Data')

In [139]:

# sql_conn = sqlite3.connect('world_soccer_database.sqlite') # soccer database containing over 100,000 match results which is updated monthly

# Match_TeamStats = pd.read_sql_query("SELECT * FROM football_data", sql_conn)

# # parsing betting odds variables from analysis
# drop_columns = ["B365H","B365D","B365A","BSH","BSD","BSA","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA",
# "PSH", "PH","PSD", "PD","PSA", "PA","SOH","SOD","SOA","SBH","SBD","SBA","SJH","SJD","SJA","SYH","SYD","SYA","VCH",
# "VCD","VCA","WHH","WHD","WHA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","MaxH","MaxD","MaxA",
# "AvgH","AvgD","AvgA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","GB>2.5","GB<2.5","B365>2.5","B365<2.5",
# "P>2.5","P<2.5","Max>2.5","Max<2.5","Avg>2.5","Avg<2.5","BbAH","BbAHh","AHh","BbMxAHH","BbAvAHH","BbMxAHA",
# "BbAvAHA","GBAHH","GBAHA","GBAH","LBAHH","LBAHA","LBAH","B365AHH","B365AHA","B365AH","PAHH", "AvgC<2.5",
# "PAHA","MaxAHH","MaxAHA","AvgAHH","AvgAHA","WHCH","WHCD","WHCA", "VCCH", "VCCA", "PSCH", "PSCD", "PCAHH", "PCAHA", "PC>2.5", "PC<2.5", "MaxCH", "MaxCAHH", "MaxCD",
# "MaxCAHA", "MaxCA", "MaxC>2.5", "MaxC<2.5", "IWCH", "IWCD", "IWCA", "AvgC>2.5","AvgCA","AvgCAHA","AvgCAHH",
# "AvgCD","AvgCH","B365C<2.5","B365C>2.5","B365CA","B365CAHA","B365CAHH","B365CD","B365CH","BWCA", "BWCD", "BWCH", "VCCD", "PSCA"] 

# Match_TeamStats.drop(drop_columns, inplace=True, axis =1) # removing betting odds from dataframe





In [140]:
# Match_TeamStats = Match_TeamStats[Match_TeamStats['Datetime'].str.contains('2017|2018')]

In [141]:
Matches["date"] = pd.to_datetime(Matches['dateutc']).dt.date # create a date column for Matches dataframe

In [142]:
Matches["time"] = pd.to_datetime(Matches['dateutc']).dt.time # create a time column for Matches dataframe

In [143]:
# Prelim Dimensionality Reduction
cols2Drop = ['status', 'roundId', 'gameweek', 'dateutc', 'label', 'referees', 'homeManagerId', 'seasonId']
Matches.drop(cols2Drop, inplace=True, axis =1)
Matches.drop(['winner', 'date', 'time'], inplace = True, axis =1)

In [144]:
Matches.rename(columns={'wyId':'matchId'}, inplace=True)

In [145]:
Matches = Matches.fillna(0) # fill bench7players with Ids = 0 in order to prevent program crashing

In [146]:
# Match_TeamStats['date'] = pd.to_datetime(Match_TeamStats['Datetime']).dt.date

In [147]:
# # Get Referee and Home/Away Team Information
# Match_TeamStats.merge(Matches, how = "inner", indicator = False)

In [148]:
# Matches = Match_TeamStats.merge(Matches, how = "inner", indicator = False)

In [149]:
# Matches.drop(['gameweek', 'seasonId', 'dateutc', 'label', 'Season', 'status', 'roundId'], inplace= True, axis =1)

In [150]:
# Matches.drop(['Datetime', 'Div','FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'ABP', 'AC',
#        'AF', 'AFKC', 'AHCh', 'AHW', 'AO', 'AR', 'AS', 'AST', 'AT', 'AY',
#        'Attendance', 'Date', 'HBP', 'HC', 'HF', 'HFKC', 'HHW', 'HO', 'HR',
#        'HS', 'HST', 'HT', 'HY', 'Time', 'referees'], inplace= True, axis =1)

### Events Manipulation for Match/Player Stats and then combine with existing Match(test) dataframe from aboves

In [151]:
# Group by and events summarization

In [152]:
Events_England['subEventName'].unique()

array(['Simple pass', 'High pass', 'Head pass', 'Air duel',
       'Ground loose ball duel', 'Smart pass', 'Launch',
       'Ground defending duel', 'Ground attacking duel', 'Foul',
       'Free Kick', 'Cross', 'Shot', 'Reflexes', 'Touch', 'Clearance',
       'Ball out of the field', 'Throw in', 'Goal kick', 'Corner',
       'Goalkeeper leaving line', 'Hand pass', 'Acceleration',
       'Save attempt', '', 'Free kick cross', 'Free kick shot',
       'Hand foul', 'Violent Foul', 'Protest', 'Whistle',
       'Late card foul', 'Out of game foul', 'Penalty', 'Time lost foul',
       'Simulation'], dtype=object)

In [153]:
# Decroos Github
tags = [
    (101, "goal"),
    (102, "own_goal"),
    (301, "assist"),
    (302, "key_pass"),
    (1901, "counter_attack"),
    (401, "left_foot"),
    (402, "right_foot"),
    (403, "head/body"),
    (1101, "direct"),
    (1102, "indirect"),
    (2001, "dangerous_ball_lost"),
    (2101, "blocked"),
    (801, "high"),
    (802, "low"),
    (1401, "interception"),
    (1501, "clearance"),
    (201, "opportunity"),
    (1301, "feint"),
    (1302, "missed_ball"),
    (501, "free_space_right"),
    (502, "free_space_left"),
    (503, "take_on_left"),
    (504, "take_on_right"),
    (1601, "sliding_tackle"),
    (601, "anticipated"),
    (602, "anticipation"),
    (1701, "red_card"),
    (1702, "yellow_card"),
    (1703, "second_yellow_card"),
    (1201, "position_goal_low_center"),
    (1202, "position_goal_low_right"),
    (1203, "position_goal_mid_center"),
    (1204, "position_goal_mid_left"),
    (1205, "position_goal_low_left"),
    (1206, "position_goal_mid_right"),
    (1207, "position_goal_high_center"),
    (1208, "position_goal_high_left"),
    (1209, "position_goal_high_right"),
    (1210, "position_out_low_right"),
    (1211, "position_out_mid_left"),
    (1212, "position_out_low_left"),
    (1213, "position_out_mid_right"),
    (1214, "position_out_high_center"),
    (1215, "position_out_high_left"),
    (1216, "position_out_high_right"),
    (1217, "position_post_low_right"),
    (1218, "position_post_mid_left"),
    (1219, "position_post_low_left"),
    (1220, "position_post_mid_right"),
    (1221, "position_post_high_center"),
    (1222, "position_post_high_left"),
    (1223, "position_post_high_right"),
    (901, "through"),
    (1001, "fairplay"),
    (701, "lost"),
    (702, "neutral"),
    (703, "won"),
    (1801, "accurate"),
    (1802, "not_accurate"),
]

In [154]:
tags = dict(tags)

In [155]:
# ## I have kept this cell encase we need to impute integers for ML algorithms as oppose to strings
# event_tag_ids = []
# for ids in  list(trial[0:10]['tags']):
#     event_tag_ids.append(list(map(lambda y: y['id'], ids)))
# trial['event_tag_ids'] = event_tag_ids

In [156]:
# Store event type tags ids in new column
def event_tags(event_df):
    event_tags = []
    for ids in list(event_df['tags']):
        event_tags.append(list(map(lambda y: tags[y['id']], ids)))

    event_df['event_tags'] = event_tags
    return event_df

In [157]:
def RunningTime(event_df):
    # Solution to convert second half time into 
    secs_to_add = list(event_df[event_df['matchPeriod']=="1H"].groupby('matchId').tail(1)['eventSec'])
    match_ids = list(event_df["matchId"].unique())

    for idx in range(0, len(match_ids)):
        event_df['eventSec'] = list(np.where(
           (event_df['matchId'] == match_ids[idx]) & (event_df['matchPeriod'] == "2H") , event_df['eventSec'] + secs_to_add[idx], event_df['eventSec']
           ))
    return event_df


In [158]:
# Clean the result of the action
def Result(event_df):
    result = []
    for tag in event_df["event_tags"]:
        if "accurate" in tag:
            result.append("Accurate")
        elif "not_accurate" in tag:
            result.append("Inaccurate")
        else:
            result.append("")
    event_df["Result"] = result 
    return event_df

In [159]:
# Separating description from success/failure of an action (event)
def event_description(event_df):
    subEventDescr = []
    for tag in event_df["event_tags"]:
        descr = " ".join(tag[:-1])
        if descr != "":
            subEventDescr.append(descr)
        else:
            subEventDescr.append("generic play")
    event_df["subEventDescription"] = subEventDescr
    event_df = event_df[(event_df["event_tags"].str.len() != 0) & (event_df.subEventName != 'Ball out of the field')  & (event_df.subEventName != 'Goal kick')  ]
    event_df = event_df[event_df["subEventName"] != "Throw in"] # remove throw-ins as a relevant feature among successful teams for simpler analysis, unless its Rory Delap!
    return event_df

In [160]:
# # Separating description from success/failure of an action (event)
# def event_description(event_df):
#     subEventDescr = []
#     for tag in event_df["event_tags"]:
#         descr = " ".join(tag[:-1])
#         if descr != "":
#             subEventDescr.append(descr)
#         else:
#             subEventDescr.append("generic play")
#     event_df["subEventDescription"] = subEventDescr
#     event_df = event_df[(event_df["event_tags"].str.len() != 0) & (event_df.subEventName != 'Ball out of the field')  & (event_df.subEventName != 'Goal kick') & (event_df.subEventName != 'Touch')   ]
#     event_df = event_df[event_df["subEventName"] != "Throw in"] # remove throw-ins as a relevant feature among successful teams for simpler analysis, unless its Rory Delap!
#     return event_df

In [164]:
#### I prefer the segmented format of the cell below for locations but we will keep just encase***
# start, end = [], []
# for pos1, pos2 in trial['positions']:
#     start.append(pos1)
#     end.append(pos2)

# trial["StartLoc"] = start
# trial["EndLoc"] = end

In [165]:
collections.Counter(list(map(lambda x: len(x), Events_England['positions'])))

Counter({2: 643032, 1: 118})

In [168]:
# xStart,xEnd,yStart,yEnd = [], [], [], []
# for pos in Events_England["positions"][0:10]:
#     if len(pos) == 2:
#         xStart.append(pos[0]['x'])
#         xEnd.append(pos[1]['x'])
#         yStart.append(pos[0]['y'])
#         yEnd.append(pos[1]['y'])
#     else:
#         xStart.append(pos[0]['x'])
#         xEnd.append(pos[0]['x'])
#         yStart.append(pos[0]['y'])
#         yEnd.append(pos[0]['y'])

In [169]:

def Locations(event_df):

    xStart,xEnd,yStart,yEnd = [], [], [], []
    for pos in event_df["positions"]:
        if len(pos) == 2:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[1]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[1]['y'])
        else:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[0]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[0]['y'])


    event_df['xStart'], event_df['xEnd'], event_df['yStart'], event_df['yEnd'] = xStart, xEnd, yStart, yEnd
    return event_df

In [170]:
#Events_England = Locations(Events_England)

In [171]:
#Events_England.drop(['eventId', 'positions', 'id', "event_tags", "tags", "eventId", "eventName"], inplace = True, axis = 1)

In [172]:
#Events_England = Events_England[Events_England.subEventName != ""]

In [197]:
# function to transform an event data frame
def event_df_transform(event_df):
    event_df = event_tags(event_df)
    event_df = Result(event_df)
    event_df = event_description(event_df)
    event_df = Locations(event_df)
    event_df.drop(['positions', 'id', "event_tags", "tags", "eventName"], inplace = True, axis = 1)
    event_df = event_df[event_df.subEventName != ""]
    event_df = event_df[event_df['playerId']!= 0]
    event_df = RunningTime(event_df)
    event_df['attackMetres'] = event_df['xEnd'] - event_df['xStart']
    return event_df

# stopped remove eventId
    

In [198]:
# Transform event_dfs

Events_France = event_df_transform(Events_France)
Events_Spain = event_df_transform(Events_Spain)
Events_Germany = event_df_transform(Events_Germany)
Events_Italy = event_df_transform(Events_Italy)
Events_England = event_df_transform(Events_England)



In [81]:
# Chris Jester-Young https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-values
# def isNaN(num):
#     return num != num

In [82]:
# this function fills in NaN values with correct teamId from the Matches df
# def missing_teams(x):
#     if np.isnan(x['team2']):
#         row = Matches[Matches['matchId'] == x["matchId"]]
#         if row['homeTeamId'].astype(int) == x['team1']:
#             return int(row['awayTeamId'])
#         else:
#             return int(row['homeTeamId'])
#     else:
#         return x['team2']

In [83]:
### remove events where playerId equals zero
## There are two instances where goals are scored so we will handle these first and assign to correct players as these are important events
# we can attempt to handle these into the real player sequences if we have time at the end of the project


In [84]:
# refs = list(Matches[Matches['matchId']==2499719].loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().index)
# ids = [int(value) for value in Matches[Matches['matchId']==2499719].loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().values]
# test = pd.DataFrame(zip(ids, refs), columns = ['playerId', 'playerRefs'])
# test = test.merge(Players[["firstName", "lastName", "wyId", "Position", 'currentTeamId']], how = 'inner', right_on = 'wyId', left_on = 'playerId')
# test.head()

In [85]:
# refs = list(Matches.loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().index)
# ids = [int(value) for value in Matches.loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().values]
# test = pd.DataFrame(zip(ids, refs), columns = ['playerId', 'playerRefs'])
# test = test.merge(Players[["firstName", "lastName", "wyId", "Position", 'currentTeamId']], how = 'inner', right_on = 'wyId', left_on = 'playerId')
# test

In [86]:
# pt1 = pd.pivot_table(trial, values = 'xStart', index='matchId', columns = 'playerRefs').reset_index().add_prefix('xPos_')
# pt1 = pt1.rename(columns = {'xPos_matchId': "matchId"})
# pt2 = pd.pivot_table(trial, values = 'yStart', index='matchId', columns = 'playerRefs').reset_index().add_prefix('yPos_')
# pt2 = pt2.rename(columns = {'yPos_matchId': "matchId"})
# pt = pd.merge(pt1, pt2, on="matchId")
# pt

In [199]:
# function to determine average player coordinates on pitch across all possessions in a match
def playerPosition(event_df):
    df = event_df.groupby(['matchId','playerId']).agg({'xStart': ['mean'], 'yStart': ['mean']}).reset_index()
    df.columns = ["matchId", "playerId", "xStart", "yStart"]
    return df

In [200]:
# find positions for all players for big 5 Euro leagues
a = playerPosition(Events_England)
b = playerPosition(Events_France)
c = playerPosition(Events_Italy)
d = playerPosition(Events_Spain)
e = playerPosition(Events_Germany)

position_df =  pd.concat([a,b,c,d,e])

In [201]:
position_df.describe()

,matchId,playerId,xStart,yStart
count,5.051800e+04,50518.000000,50518.000000,50518.000000
mean,2.532662e+06,97586.823152,48.750253,50.103268
std,3.312466e+04,123688.451486,15.338131,20.357378
min,2.499719e+06,33.000000,2.928571,1.500000
25%,2.500766e+06,12242.000000,38.192618,35.303015
50%,2.516895e+06,25747.000000,50.104031,50.051632
75%,2.565854e+06,173214.000000,60.634498,65.000000
max,2.576338e+06,568583.000000,97.000000,100.000000


In [202]:
# This dataframe is used relationally to fill correct team as events represent 2018 team, while player
a = Events_England[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
b = Events_France[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
c = Events_Italy[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
d = Events_Spain[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
e = Events_Germany[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
p_refs = pd.concat([a,b,c,d,e])

In [203]:
Player_Aggs = Players[['shortName','Position', 'playerId','weight','height',  'foot' ]]

In [204]:
# add ave field locations to players
Player_Aggs = pd.merge(Player_Aggs, position_df, how = 'right', on= 'playerId').sort_values('matchId')
Player_Aggs = pd.merge(Player_Aggs, p_refs, on = ['matchId', 'playerId'])
Player_Aggs = pd.merge(Player_Aggs, Teams[['teamId', 'name']], on = 'teamId')

In [205]:
Player_Aggs = pd.merge(Player_Aggs, Matches[['matchId', 'homeTeamId', 'awayTeamId']], on = 'matchId')
Player_Aggs.loc[Player_Aggs.teamId == Player_Aggs.homeTeamId, 'homeAway'] = "home"
Player_Aggs.loc[Player_Aggs.teamId == Player_Aggs.awayTeamId, 'homeAway'] = "away"


In [206]:
### unique entry dataframe for player gameTime at any point in a match by summarizing streams

def playerGameTime(event_df):
    df = event_df.groupby(['matchId','playerId'])['eventSec'].agg(['max', 'min']).reset_index()
    df['gameTime (s)'] = (df['max'] - df['min']) # game time in seconds in seconds
    df.drop(['max', 'min'], inplace=True, axis=1)
    return df

# derive gameTime per player and store in summary table
a = playerGameTime(Events_England)
b = playerGameTime(Events_France)
c = playerGameTime(Events_Italy)
d = playerGameTime(Events_Spain)
e = playerGameTime(Events_Germany)

playingTime = pd.concat([a,b,c,d,e])


In [207]:
# convenience function for event_df parsing
def parseEvent_df(func):
    a = func(Events_England)
    b = func(Events_France)
    c = func(Events_Italy)
    d = func(Events_Spain)
    e = func(Events_Germany)
    
    return pd.concat([a,b,c,d,e])

In [247]:
# function for adding new feature column to Player_Aggs df
def alter_Player_Aggs(new_feature_function, new_feature, PA_df):

    df = parseEvent_df(new_feature_function)
    PA_df = pd.merge(PA_df, df, how= 'left', on =['playerId', 'matchId'])
    PA_df[new_feature] = PA_df[new_feature].fillna(0)

    return PA_df


In [208]:
Player_Aggs = pd.merge(Player_Aggs, playingTime, how = 'left', on = ['playerId', 'matchId'])

In [209]:
adv_stats = ["aerialDuelsWon",  "defensiveDuelsWon",  "directFreeKicksOnTarget",  "duelsWon",  "fieldAerialDuelsWon",  "gkAerialDuelsWon",  "gkSaves",  "gkSuccessfulExits",  "goalConversion",  "headShotsOnTarget",  "newDefensiveDuelsWon",  "newDuelsWon",  "newOffensiveDuelsWon",  "newSuccessfulDribbles",  "offensiveDuelsWon",  "penaltiesConversion",  "shotsOnTarget",  "successfulBackPasses",  "successfulCrosses",  "successfulDribbles",  "successfulForwardPasses",  "successfulKeyPasses",  "successfulLateralPasses",  "successfulLinkupPlays",  "successfulLongPasses",  "successfulPasses",  "successfulPassesToFinalThird",  "successfulShotAssists",  "successfulSmartPasses",  "successfulThroughPasses",  "successfulTouchInBox",  "successfulVerticalPasses",  "win",  "yellowCardsPerFoul",  "accelerations",  "aerialDuels",  "aerialDuelsWon",  "assists",  "attackingActions",  "backPasses",  "cleanSheets",  "concededGoals",  "corners",  "crosses",  "dangerousOpponentHalfRecoveries",  "dangerousOwnHalfLosses",  "defensiveActions",  "defensiveDuels",  "defensiveDuelsWon",  "directFreeKicks",  "directFreeKicksOnTarget",  "directRedCards",  "dribbles",  "duels",  "duelsWon",  "fieldAerialDuels",  "fieldAerialDuelsWon",  "forwardPasses",  "fouls",  "freeKicks",  "freeKicksOnTarget",  "gkAerialDuels",  "gkAerialDuelsWon",  "gkExits",  "gkGoalKicks",  "gkGoalKicksSuccess",  "gkSaves",  "gkSuccessfulExits",  "goals",  "headShots",  "interceptions",  "keyPasses",  "lateralPasses",  "linkupPlays",  "longGoalKicks",  "longPasses",  "looseBallDuels",  "looseBallDuelsWon",  "losses",  "matches",  "matchesTagged",  "missedBalls",  "newDefensiveDuelsWon",  "newDuelsWon",  "newOffensiveDuelsWon",  "newSuccessfulDribbles",  "offensiveDuels",  "offensiveDuelsWon",  "offsides",  "opponentHalfRecoveries",  "opponentOffsides",  "ownHalfLosses",  "passes",  "passesToFinalThird",  "penalties",  "pressingDuels",  "pressingDuelsWon",  "progressiveRun",  "receivedPass",  "recoveries",  "redCards",  "shortGoalKicks",  "shotAssists",  "shotOnTargetAssists",  "shots",  "shotsAgainst",  "smartPasses",  "successfulAttackingActions",  "successfulBackPasses",  "successfulCrosses",  "successfulDefensiveActions",  "successfulDribbles",  "successfulForwardPasses",  "successfulKeyPasses",  "successfulLateralPasses",  "successfulLinkupPlays",  "successfulLongPasses",  "successfulPasses",  "successfulPassesToFinalThird",  "successfulPenalties",  "successfulSmartPasses",  "successfulThroughPasses",  "successfulVerticalPasses",  "throughPasses",  "touchInBox",  "verticalPasses",  "xgShot",  "xgShotAgainst",  "yellowCards",  "accelerations",  "aerialDuels",  "assists",  "attackingActions",  "backPasses",  "ballLosses",  "ballRecoveries",  "clearances",  "corners",  "counterpressingRecoveries",  "crosses",  "dangerousOpponentHalfRecoveries",  "dangerousOwnHalfLosses",  "defensiveActions",  "defensiveDuels",  "defensiveDuelsWon",  "directFreeKicks",  "directFreeKicksOnTarget",  "directRedCards",  "dribbleDistanceFromOpponentGoal",  "dribbles",  "dribblesAgainst",  "dribblesAgainstWon",  "duels",  "duelsWon",  "fieldAerialDuels",  "fieldAerialDuelsWon",  "forwardPasses",  "fouls",  "foulsSuffered",  "freeKicks",  "freeKicksOnTarget",  "gkAerialDuels",  "gkAerialDuelsWon",  "gkConcededGoals",  "gkExits",  "gkSaves",  "gkShotsAgainst",  "gkSuccessfulExits",  "goalKicks",  "goalKicksLong",  "goalKicksShort",  "goals",  "headShots",  "interceptions",  "keyPasses",  "lateralPasses",  "linkupPlays",  "longPassLength",  "longPasses",  "looseBallDuels",  "looseBallDuelsWon",  "missedBalls",  "newDefensiveDuelsWon",  "newDuelsWon",  "newOffensiveDuelsWon",  "newSuccessfulDribbles",  "offensiveDuels",  "offensiveDuelsWon",  "offsides",  "opponentHalfRecoveries",  "ownHalfLosses",  "passLength",  "passes",  "passesToFinalThird",  "penalties",  "progressivePasses",  "progressiveRun",  "receivedPass",  "redCards",  "secondAssists",  "shotAssists",  "shotOnTargetAssists",  "shots",  "shotsBlocked",  "shotsOnTarget",  "slidingTackles",  "smartPasses",  "successfulAttackingActions",  "successfulBackPasses",  "successfulCrosses",  "successfulDefensiveAction",  "successfulDribbles",  "successfulForwardPasses",  "successfulGoalKicks",  "successfulKeyPasses",  "successfulLateralPasses",  "successfulLinkupPlays",  "successfulLongPasses",  "successfulPasses",  "successfulPassesToFinalThird",  "successfulPenalties",  "successfulProgressivePasses",  "successfulSlidingTackles",  "successfulSmartPasses",  "successfulThroughPasses",  "successfulVerticalPasses",  "thirdAssists",  "throughPasses",  "touchInBox",  "verticalPasses",  "xgAssist",  "xgSave",  "xgShot",  "yellowCards",  "avgPossessionDuration",  "minutesOfPossession1-15",  "minutesOfPossession106-120",  "minutesOfPossession16-30",  "minutesOfPossession31-45",  "minutesOfPossession46-60",  "minutesOfPossession61-75",  "minutesOfPossession76-90",  "minutesOfPossession91-105",  "possession1-15",  "possession106-120",  "possession16-30",  "possession31-45",  "possession46-60",  "possession61-75",  "possession76-90",  "possession91-105",  "possessionNumber",  "possessionPercent",  "purePossessionTime",  "reachingOpponentBox",  "reachingOpponentHalf",  "avgPossessionDuration",  "minutesOfPossession1-15",  "minutesOfPossession106-120",  "minutesOfPossession16-30",  "minutesOfPossession31-45",  "minutesOfPossession46-60",  "minutesOfPossession61-75",  "minutesOfPossession76-90",  "minutesOfPossession91-105",  "possession1-15",  "possession106-120",  "possession16-30",  "possession31-45",  "possession46-60",  "possession61-75",  "possession76-90",  "possession91-105",  "possessionNumber",  "possessionPercent",  "purePossessionTime",  "reachingOpponentBox",  "reachingOpponentHalf",  "assists",  "avgPassLength",  "avgPassToFinalThirdLength",  "backPasses",  "backPassesSuccessful",  "crossesBlocked",  "crossesFromLeftFlank",  "crossesFromLeftFlankSuccessful",  "crossesFromRightFlank",  "crossesFromRightFlankSuccessful",  "crossesHigh",  "crossesLow",  "crossesSuccessful",  "crossesTotal",  "deepCompletedPasses",  "deepCompletedPassesSuccessful",  "forwardPasses",  "forwardPassesSuccessful",  "keyPasses",  "keyPassesSuccessful",  "lateralPasses",  "lateralPassesSuccessful",  "longPasses",  "longPassesSuccessful",  "matchTempo",  "passToFinalThirds",  "passToFinalThirdsSuccessful",  "passToPenaltyAreas",  "passToPenaltyAreasSuccessful",  "passes",  "passesSuccessful",  "progressivePasses",  "progressivePassesSuccessful",  "shortMediumPasses",  "shortMediumPassesSuccessful",  "shotAssists",  "smartPasses",  "smartPassesSuccessful",  "throughPasses",  "throughPassesSuccessful",  "verticalPasses",  "verticalPassesSuccessful",  "corners",  "cornersWithShot",  "counterAttacks",  "freeKicks",  "freeKicksWithShot",  "positionalAttack",  "positionalWithShots",  "total",  "withShots",  "clearances",  "interceptions",  "ppda",  "tackles",  "aerialDuels",  "aerialDuelsSuccessful",  "challengeIntensity",  "defensiveDuels",  "defensiveDuelsSuccessful",  "dribbles",  "dribblesSuccessful",  "duels",  "duelsSuccessful",  "groundDuels",  "groundDuelsSuccessful",  "looseBallDuels",  "looseBallDuelsSuccessful",  "offensiveDuels",  "offensiveDuelsSuccessful"]
adv_stats = list(set(adv_stats))

In [210]:
Player_Aggs.head()

,shortName,Position,playerId,weight,height,foot,matchId,xStart,yStart,teamId,name,homeTeamId,awayTeamId,homeAway,gameTime (s)
0,M. \u00d6zil,MID,3319,76.0,180.0,left,2499719,63.549020,43.901961,1609,Arsenal,1609,1631,home,5774.608975
1,A. Ramsey,MID,7870,76.0,183.0,right,2499719,61.857143,48.333333,1609,Arsenal,1609,1631,home,1670.013269
2,T. Walcott,FWD,7879,68.0,176.0,right,2499719,50.250000,86.500000,1609,Arsenal,1609,1631,home,560.259270
3,P. \u010cech,GKP,7882,90.0,196.0,left,2499719,10.380952,40.571429,1609,Arsenal,1609,1631,home,5556.288749
4,R. Holding,DEF,370224,75.0,189.0,right,2499719,35.337500,76.550000,1609,Arsenal,1609,1631,home,4103.863725


In [211]:
# result is event_df
def player_goals(result):

    conditions = [
            (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Success"))),
            (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Failure") & (result['subEventName'] == "Shot"))),
            (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Failure") & (result['subEventName'] != "Shot")))
            ]

    # create a list of the values we want to assign for each condition
    values = [1, 1, -1]

    # create a new column and us np.select to assign values to it using our lists as arguments
    result['Goal_Value'] = np.select(conditions, values, default = 0)
    
    return result
    
    

In [228]:
### Attack Metres stat (how many forward metres achieved through events)
def successAttackMetres(event_df):

    df = event_df.loc[event_df.Result == "Accurate" ].groupby(['matchId','playerId'])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "AccurateAttackMetres"})

    return df

def failAttackMetres(event_df):

    df = event_df.loc[event_df.Result == "Inaccurate" ].groupby(['matchId','playerId'])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "InaccurateAttackMetres"})

    return df


In [291]:
# adding in total meterage of accurate and inaccurate movements by players in the vertical plane towards the goal
successAMetres = parseEvent_df(successAttackMetres)
failedAMetres = parseEvent_df(failAttackMetres)
Player_Aggs = pd.merge(Player_Aggs, successAMetres, how= 'left', on =['playerId', 'matchId'])
Player_Aggs = pd.merge(Player_Aggs, failedAMetres, how = 'left', on =['playerId', 'matchId'])
Player_Aggs.AccurateAttackMetres = Player_Aggs.AccurateAttackMetres.fillna(0)
Player_Aggs.InaccurateAttackMetres = Player_Aggs.InaccurateAttackMetres.fillna(0)

In [290]:
# Player_Aggs =  Player_Aggs.iloc[:, :-2]

In [301]:
def find_goals(event_df):

    # Create a column to recognize when a goal is scored
    conditions = [
        (((event_df['subEventDescription'].str.match('goal ')) & (event_df['Result'] == "Accurate"))),
        (((event_df['subEventDescription'].str.match('goal ')) & (event_df['Result'] == "Inaccurate") & (event_df['subEventName'] == "Shot"))),
        (((event_df['subEventDescription'].str.match('goal ')) & (event_df['Result'] == "Inaccurate") & (event_df['subEventName'] != "Shot")))
        ]

    # create a list of the values we want to assign for each condition
    values = [1, 1, -1]

    # create a new column and use np.select to assign values to it using our lists as arguments
    event_df['Goal_Value'] = np.select(conditions, values, default = 0)
    
    return event_df.groupby(['playerId', "matchId"]).sum().reset_index()[['playerId', 'matchId', 'Goal_Value']]

In [251]:
goals = parseEvent_df(find_goals)
Player_Aggs = pd.merge(Player_Aggs, goals, how= 'left', on =['playerId', 'matchId'])

##### Explore Feature

In [307]:
Events_England.subEventName.unique()

array(['Simple pass', 'High pass', 'Head pass', 'Air duel',
       'Ground loose ball duel', 'Smart pass', 'Launch',
       'Ground defending duel', 'Ground attacking duel', 'Cross', 'Shot',
       'Reflexes', 'Touch', 'Clearance', 'Free Kick', 'Corner',
       'Hand pass', 'Acceleration', 'Save attempt', 'Free kick cross',
       'Foul', 'Free kick shot', 'Hand foul', 'Violent Foul', 'Protest',
       'Goalkeeper leaving line', 'Late card foul', 'Out of game foul',
       'Penalty', 'Time lost foul', 'Simulation'], dtype=object)

In [ ]:
# goalkeeping, dribbles, accelerations, crosses

In [486]:
Events_England[(Events_England.subEventName == 'Touch')]


,eventId,subEventName,playerId,matchId,teamId,matchPeriod,eventSec,subEventId,Result,subEventDescription,xStart,xEnd,yStart,yEnd,attackMetres,Goal_Value
1846,7,Touch,8325,2499720,1625,1H,184.085987,72,,generic play,92,82,43,43,-10,0
1940,7,Touch,8277,2499720,1625,1H,457.280577,72,,opportunity,95,98,62,56,3,0
2150,7,Touch,340386,2499720,1625,1H,1200.160927,72,,generic play,92,100,58,73,8,0
2171,7,Touch,9380,2499720,1625,1H,1402.770829,72,,generic play,91,88,62,57,-3,0
2229,7,Touch,340386,2499720,1625,1H,1584.375157,72,,generic play,88,92,57,63,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641133,7,Touch,3928,2500097,1631,2H,3713.191990,72,,generic play,87,87,40,48,0,0
641257,7,Touch,36,2500097,1624,2H,4190.506691,72,,generic play,91,89,33,34,-2,0
641545,7,Touch,8717,2500097,1624,2H,5368.236240,72,,generic play,94,78,51,88,-16,0
642139,7,Touch,265318,2500098,1633,1H,1831.284544,72,,generic play,95,98,30,31,3,0


#### Tackles & Ground Duels

In [496]:

# Sliding tackles won
def slidingTacklesWon(event_df):
    return event_df[(event_df.subEventDescription.str.contains("sliding_tackle won"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='SlidingTacklesWon')

# Sliding tackles lost
def slidingTacklesLost(event_df):
    return event_df[(event_df.subEventDescription.str.contains("sliding_tackle lost"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='SlidingTacklesLost')

# Ground Defending Duels Won
def GroundDuelsWon(event_df):
    return event_df[(event_df.subEventName=="Ground defending duel") & event_df.subEventDescription.str.contains("won")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='GroundDefDuelsWon')

# Ground Defending Duels Lost
def GroundDuelsLost(event_df):
    return event_df[(event_df.subEventName=="Ground defending duel") & event_df.subEventDescription.str.contains("lost")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='GroundDefDuelsLost')

# Ground Defensive Anticipations
def GroundDefAnticipations(event_df):
    return event_df[(event_df.subEventName=="Ground defending duel") & event_df.subEventDescription.str.contains("anticipated")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='DefensiveAnticipations')

# Ground Loose Balls Won
def GroundLooseBallsWon(event_df):
    return event_df[(event_df.subEventName == 'Ground loose ball duel') &(event_df.subEventDescription.str.contains('won'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='GroundLooseBallsWon')

# Ground Loose Balls Lost
def GroundLooseBallsLost(event_df):
    return event_df[(event_df.subEventName == 'Ground loose ball duel') &(event_df.subEventDescription.str.contains('lost'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='GroundLooseBallsLost')

# Ground attacking duels won
def GroundAttackDuelsWon(event_df):
    return event_df[(event_df.subEventName == 'Ground attacking duel') &(event_df.subEventDescription.str.contains('won'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='GroundAttackDuelsWon')

# Ground attacking duels lost
def GroundAttackDuelsLost(event_df):
    return event_df[(event_df.subEventName == 'Ground attacking duel') &(event_df.subEventDescription.str.contains('lost'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='GroundAttackDuelsLost')

# Offensive Touches in box
def OffTouchesBox(event_df):
    return event_df[(event_df.subEventName == 'Touch')
              & (event_df.xStart > 82)
              & (event_df.yStart < 80)
              & (event_df.yStart > 20)][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='OffensiveTouchesBox')




In [497]:
# 
Player_Aggs = alter_Player_Aggs(slidingTacklesWon, "SlidingTacklesWon", Player_Aggs)
Player_Aggs = alter_Player_Aggs(slidingTacklesLost, "SlidingTacklesLost", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundDuelsWon, "GroundDefDuelsWon", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundDuelsLost, "GroundDefDuelsLost", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundDefAnticipations, "DefensiveAnticipations", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundLooseBallsWon, "GroundLooseBallsWon", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundLooseBallsLost, "GroundLooseBallsLost", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundAttackDuelsWon, "GroundAttackDuelsWon", Player_Aggs)
Player_Aggs = alter_Player_Aggs(GroundAttackDuelsLost, "GroundAttackDuelsLost", Player_Aggs)
Player_Aggs = alter_Player_Aggs(OffTouchesBox, "OffensiveTouchesBox", Player_Aggs)

Player_Aggs = Player_Aggs.drop_duplicates()


#### Crosses, Dribbles, Accelerations

#### Shots & Goals

In [458]:
# Shots on Target
def ShotsOnTarget(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.Result=="Accurate")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='ShotsOnTarget')

# Shots off Target
def ShotsOffTarget(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.Result=="Inaccurate")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='ShotsOffTarget')

# Shot opportunity
def shot_opportunities(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.contains("opportunity"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='ShotOpportunities')

# Accurate free kick shots
def accFreeKicks(event_df):
    return event_df[(event_df.subEventName=="Free kick shot") & (event_df.Result=="Accurate")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='FreeKickShotAcc')

# Inaccurate free kick shots
def InaccFreeKicks(event_df):
    return event_df[(event_df.subEventName=="Free kick shot") & (event_df.Result=="Inaccurate")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='FreeKickShotInacc')

# Scored penalty
def PenaltyGoal(event_df):
    return event_df[(event_df.subEventName=="Penalty") & (event_df.Result=="Accurate")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='PenaltyConverted')

# Did not score penalty
def PenaltyNotGoal(event_df):
    return event_df[(event_df.subEventName=="Penalty") & (event_df.Result=="Inaccurate")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='PenaltyNotConverted')

# Goal scored with head/body
def HeaderGoal(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("head"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='Header/Body_Goal')

# Goal scored with right foot
def RightFootGoal(event_df): 
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("right_foot"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='RightFootGoal')

# Goal scored with left foot
def LeftFootGoal(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("left_foot"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='LeftFootGoal')

# Counter Attack Goal
def CounterAttackGoal(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("counter_attack"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='CounterAttackGoal')

# Interception Goal
def InterceptionGoal(event_df):
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("interception"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InterceptionGoal')

# Goal positions
def GoalPositions(event_df, pos):
    return event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
                  & (event_df.subEventDescription.str.contains(pos))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='Goal_'+pos)
    
def OwnGoals(event_df):
    return event_df[event_df.subEventDescription.str.contains("own")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='OwnGoals')
    



In [414]:
# Get positional zones of goals for players

possies = ['low_left', 'mid_left', 'high_left', 'low_center', 'mid_center', 'high_center', 'low_right', 'mid_right', 'high_right']
for pos in possies:
    a = GoalPositions(Events_England, pos)
    b = GoalPositions(Events_France, pos)
    c = GoalPositions(Events_Italy, pos)
    d = GoalPositions(Events_Spain, pos)
    e = GoalPositions(Events_Germany, pos)
    df = pd.concat([a,b,c,d,e])
    Player_Aggs = pd.merge(Player_Aggs, df, how= 'left', on =['playerId', 'matchId'])
    Player_Aggs["Goal_"+pos] = test["Goal_"+pos].fillna(0)
    


In [421]:
# Patch up to remove duplicate rows
Player_Aggs = Player_Aggs.drop_duplicates()

In [459]:
# Add shot and goal summaries for players per match
Player_Aggs = alter_Player_Aggs(ShotsOnTarget, "ShotsOnTarget", Player_Aggs)
Player_Aggs = alter_Player_Aggs(ShotsOffTarget, "ShotsOffTarget", Player_Aggs)
Player_Aggs = alter_Player_Aggs(shot_opportunities, "ShotOpportunities", Player_Aggs)
Player_Aggs = alter_Player_Aggs(accFreeKicks, "FreeKickShotAcc", Player_Aggs)
Player_Aggs = alter_Player_Aggs(InaccFreeKicks, "FreeKickShotInacc", Player_Aggs)
Player_Aggs = alter_Player_Aggs(PenaltyGoal, "PenaltyConverted", Player_Aggs)
Player_Aggs = alter_Player_Aggs(PenaltyNotGoal, "PenaltyNotConverted", Player_Aggs)
Player_Aggs = alter_Player_Aggs(HeaderGoal, "Header/Body_Goal", Player_Aggs)
Player_Aggs = alter_Player_Aggs(RightFootGoal, "RightFootGoal", Player_Aggs)
Player_Aggs = alter_Player_Aggs(LeftFootGoal, "LeftFootGoal", Player_Aggs)
Player_Aggs = alter_Player_Aggs(CounterAttackGoal, "CounterAttackGoal", Player_Aggs)
Player_Aggs = alter_Player_Aggs(InterceptionGoal, "InterceptionGoal", Player_Aggs)
Player_Aggs = alter_Player_Aggs(OwnGoals, "OwnGoals", Player_Aggs)

####   Air duel

In [324]:
def AerialDuelsWon(events_df):
    return Events_England[(Events_England.subEventName=="Air duel") & (Events_England.subEventDescription.str.contains("won"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AerialDuelsWon')

def AerialDuelsLost(events_df):
    return  Events_England[(Events_England.subEventName=="Air duel") & (Events_England.subEventDescription.str.contains("lost"))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AerialDuelsLost')



In [328]:
Player_Aggs = alter_Player_Aggs(AerialDuelsWon, "AerialDuelsWon", Player_Aggs)
Player_Aggs = alter_Player_Aggs(AerialDuelsLost, "AerialDuelsLost", Player_Aggs)

#### Passing

In [285]:
# Accurate forward passes
def acc_forward_pass(event_df):
    return event_df[(event_df.subEventName=="Simple pass") 
                   & (event_df.attackMetres>0)
                   & (event_df.Result=="Accurate")
                   & (event_df.subEventDescription.str.match('generic play'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccurateForwardPasses')

# Inaccurate forward passes 
def inacc_forward_passes(event_df):
    return event_df[(event_df.subEventName=="Simple pass") 
                   & (event_df.attackMetres>0)
                   & (event_df.Result=="Inaccurate")
                   & (event_df.subEventDescription.str.match('generic play'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InaccurateForwardPasses')

# SuccessfulInterceptions ## when a player intercepts a play and makes a successful subsequent play
def success_intercept(event_df):
    return event_df[(event_df.Result == 'Accurate')
                   & (event_df.subEventDescription.str.match('interception'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='SuccessfulInterceptions')

# CounterAttackInterceptions
def CounterAttackIntercepts(event_df):
    return event_df[(event_df.subEventDescription.str.match('counter_attack interception'))].groupby(['playerId', 'matchId']).size().reset_index(name='CounterAttackIntercept')

# DangerousBallLostPassing
def dangerous_ball_lost(event_df):
    return event_df[(event_df.subEventName.str.contains("pass")) 
                   & (event_df.subEventDescription.str.match('dangerous_ball_lost'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='PassLostDangerous')

# Counter attack involvements
def CounterAttackInvolvements(event_df):
    return event_df[(event_df.subEventDescription.str.match('counter_attack')) 
                  & (event_df.Result == 'Accurate') ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='CounterAttackInvolvements')

# Total Assists
def total_assists(event_df):
    return event_df[(event_df.subEventDescription.str.contains('assist')) 
                  & (event_df.Result == 'Accurate') ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='TotalAssists')

# Inaccurate Keypasses
def inacc_key_passes(event_df):
    return event_df[(event_df.subEventDescription.str.contains('key_pass')) 
                  & (event_df.Result == "Inaccurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InaccurateKeyPasses')

# Accurate Keypasses
def acc_keypasses(event_df):
    return event_df[(event_df.subEventDescription.str.contains('key_pass')) 
                  & (event_df.Result == "Accurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccurateKeyPasses')

# Accurate Through Passes
def acc_throughs(event_df):
    return event_df[(event_df.subEventDescription.str.contains('through')) 
                  & (event_df.Result == "Accurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccurateThroughPasses')

# Inaccurate Through Passes
def inacc_throughs(event_df):
    return event_df[(event_df.subEventDescription.str.contains('through')) 
                  & (event_df.Result == "Inaccurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InaccurateThroughPasses')

# Accurate Smart Passes 
def acc_smart_passes(event_df):
    return event_df[(event_df.subEventName.str.match('Smart pass')) 
                  & (event_df.Result == "Accurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccurateSmartPasses')

# Inaccurate Smart Passes 
def inacc_smartPasses(event_df):
    return event_df[(event_df.subEventName.str.match('Smart pass')) 
                  & (event_df.Result == "Inaccurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InaccurateSmartPasses')


# Through passes into final third
def throughs_into_final(event_df):
    return event_df[(event_df.subEventDescription.str.contains('through')) 
                  & (event_df.xStart < 67 ) 
                  & (event_df.xEnd > 66 ) 
                 & (event_df.Result == "Accurate") ][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccThroughsIntoFinalThird')

# Accurate Passes into final third
def accPasses_into_final(event_df):
    return event_df[ ( event_df.subEventName.str.contains('pass'))
        & (event_df.xStart < 67 ) 
                  & (event_df.xEnd > 66) 
                 & (event_df.Result == "Accurate")
                & (event_df.Goal_Value != 1)][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccPassesIntoFinalThird')

# Accurate Passes into final third
def InaccPasses_into_final(event_df):
    return event_df[ ( event_df.subEventName.str.contains('pass'))
        & (event_df.xStart < 67 ) 
                  & (event_df.xEnd > 66) 
                 & (event_df.Result == "Inaccurate")
                & (event_df.Goal_Value != 1)][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InaccPassesIntoFinalThird')


# Accurate Passes within final third
def accPasses_within_finalThird(event_df):
    return event_df[ (event_df.subEventName.str.contains("pass") )
                   & (event_df.xStart > 66 ) 
                  & (event_df.xEnd > 66 ) 
                 & (event_df.Result == "Accurate")
                & (event_df.Goal_Value != 1)][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='AccPassesWithinFinalThird')

# Inaccurate Passes within final third   
def inaccPasses_within_finalThird(event_df):
    return event_df[ (event_df.subEventName.str.contains("pass") )
                   & (event_df.xStart > 66 ) 
                  & (event_df.xEnd > 66) 
                 & (event_df.Result == "Inaccurate")
                & (event_df.Goal_Value != 1)][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='InaccPassesWithinFinalThird')



##### Passing Summary Stats Aggregated per player per match

In [286]:
Player_Aggs = alter_Player_Aggs(inaccPasses_within_finalThird, "InaccPassesWithinFinalThird", Player_Aggs)
Player_Aggs = alter_Player_Aggs(accPasses_within_finalThird, "AccPassesWithinFinalThird", Player_Aggs)
Player_Aggs = alter_Player_Aggs(accPasses_into_final, "AccPassesIntoFinalThird", Player_Aggs)
Player_Aggs = alter_Player_Aggs(InaccPasses_into_final, "InaccPassesIntoFinalThird", Player_Aggs)
Player_Aggs = alter_Player_Aggs(throughs_into_final, "AccThroughsIntoFinalThird", Player_Aggs)
Player_Aggs = alter_Player_Aggs(inacc_smartPasses, "InaccurateSmartPasses", Player_Aggs)
Player_Aggs = alter_Player_Aggs(acc_smart_passes, "AccurateSmartPasses", Player_Aggs)
Player_Aggs = alter_Player_Aggs(inacc_throughs, "InaccurateThroughPasses", Player_Aggs)
Player_Aggs = alter_Player_Aggs(acc_throughs, "AccurateThroughPasses", Player_Aggs)
Player_Aggs = alter_Player_Aggs(acc_keypasses, "AccurateKeyPasses", Player_Aggs) 
Player_Aggs = alter_Player_Aggs(inacc_key_passes, "InaccurateKeyPasses", Player_Aggs)
Player_Aggs = alter_Player_Aggs(total_assists, "TotalAssists", Player_Aggs)
Player_Aggs = alter_Player_Aggs(CounterAttackInvolvements, "CounterAttackInvolvements", Player_Aggs) 
Player_Aggs = alter_Player_Aggs(dangerous_ball_lost, "PassLostDangerous", Player_Aggs) 
Player_Aggs = alter_Player_Aggs(CounterAttackIntercepts, "CounterAttackIntercept", Player_Aggs) 
Player_Aggs = alter_Player_Aggs(success_intercept, "SuccessfulInterceptions", Player_Aggs) 
Player_Aggs = alter_Player_Aggs(inacc_forward_passes, "InaccurateForwardPasses", Player_Aggs) 
Player_Aggs = alter_Player_Aggs(acc_forward_pass, "AccurateForwardPasses", Player_Aggs) 

#### Corners

In [137]:
def corner_assists(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.subEventDescription.str.match('assist'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='CornerAssists')

def corner_opportunity(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.subEventDescription.str.match('key_pass'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='CornerOpportunitiesCreated')

def corner_success(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.Result=="Success")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='SuccessfulCorners')

def corner_fail(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.Result=="Failure")][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='FailedCorners')

def corner_assists(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.subEventDescription.str.match('assist'))][['playerId', 'matchId']].groupby(['playerId', 'matchId']).size().reset_index(name='CornerAssists')


In [144]:
Player_Aggs = alter_Player_Aggs(corner_assists, "CornerAssists", Player_Aggs)
Player_Aggs = alter_Player_Aggs(corner_opportunity, "CornerOpportunitiesCreated", Player_Aggs)
Player_Aggs = alter_Player_Aggs(corner_success, "SuccessfulCorners", Player_Aggs)
Player_Aggs = alter_Player_Aggs(corner_fail, "FailedCorners", Player_Aggs)

In [152]:
Player_Aggs.head()

,shortName,Position,playerId,weight,height,foot,matchId,xStart,yStart,teamId,...,awayTeamId,homeAway,gameTime (s),successfulAttackYards,failedAttackYards,Goal_Value,CornerAssists,CornerOpportunitiesCreated,SuccessfulCorners,FailedCorners
0,M. \u00d6zil,MID,3319,76.0,180.0,left,2499719,63.549020,43.901961,1609,...,1631,home,5774.608975,-88.0,-282.0,0,0.0,0.0,0.0,1.0
1,Beller\u00edn,DEF,167145,74.0,178.0,right,2499719,52.695122,67.012195,1609,...,1631,home,5655.146793,-170.0,-12.0,0,0.0,0.0,0.0,0.0
2,Mohamed Elneny,MID,120339,70.0,180.0,right,2499719,50.569767,55.534884,1609,...,1631,home,4027.690742,-14.0,-33.0,0,0.0,0.0,0.0,0.0
3,G. Xhaka,MID,49876,82.0,185.0,left,2499719,57.470085,41.153846,1609,...,1631,home,5829.406096,452.0,56.0,0,1.0,1.0,6.0,1.0
4,O. Giroud,FWD,26010,88.0,192.0,left,2499719,63.466667,50.200000,1609,...,1631,home,1578.302322,-140.0,-29.0,1,0.0,0.0,0.0,0.0


In [113]:
# # Parsing player positions on field for each match in a season, and adding output to Matches summary dataframe

# for match in matches:
#     trial = trial2[trial2['matchId']== match]
#     refs = list(Matches[Matches['matchId']==match].loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().index)
#     ids = [int(value) for value in Matches[Matches['matchId']==match].loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().values]
#     test = pd.DataFrame(zip(ids, refs), columns = ['playerId', 'playerRefs'])
#     trial = pd.merge(test, trial, on="playerId")
#     pt1 = pd.pivot_table(trial, values = 'xStart', index='matchId', columns = 'playerRefs').reset_index().add_prefix('xPos_')
#     pt1 = pt1.rename(columns = {'xPos_matchId': "matchId"})
#     pt2 = pd.pivot_table(trial, values = 'yStart', index='matchId', columns = 'playerRefs').reset_index().add_prefix('yPos_')
#     pt2 = pt2.rename(columns = {'yPos_matchId': "matchId"})
#     pt = pd.merge(pt1, pt2, on="matchId")
#     temp_df = temp_df.append(pt, ignore_index = True)

# England_Matches = pd.merge(England_Matches, temp_df, on = 'matchId').fillna(0) # consider removing bench players, also consider ignoring goalkeepers






In [114]:
# Likely I will decide to deprecate this

# Create a function to aggregate stats for a match 

# stat is the string representation of stat desired
# unique_df is the unique df developed prior to represent stat grouped by player and match
# def advanced_stats(unique_df, stat, match_df):

#     matches = list(unique_df['matchId'].unique())
#     temp_df = pd.DataFrame()

#     for match in matches:
#         trial = unique_df[unique_df['matchId']== match]
#         refs = list(Matches[Matches['matchId']==match].loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().index)
#         ids = [int(value) for value in Matches[Matches['matchId']==match].loc[:,'homePlayer1':'awayBenchPlayer7'].transpose().values]
#         test = pd.DataFrame(zip(ids, refs), columns = ['playerId', 'playerRefs'])
#         trial = pd.merge(test, trial, on="playerId")
#         pt = pd.pivot_table(trial, values = stat, index='matchId', columns = 'playerRefs').reset_index().add_prefix(stat)
#         pt = pt.rename(columns = {stat + '_matchId': "matchId"})
#     #     pt2 = pd.pivot_table(trial, values = 'yStart', index='matchId', columns = 'playerRefs').reset_index().add_prefix('yPos_')
#     #     pt2 = pt2.rename(columns = {'yPos_matchId': "matchId"})
#     #     pt = pd.merge(pt1, pt2, on="matchId")
#         temp_df = temp_df.append(pt, ignore_index = True)

#     match_df = pd.merge(match_df, temp_df, on = 'matchId').fillna(0) 

#     return match_df



In [115]:

# Simple df selection to remove playerIds that = 0
#Events_England = Events_England[Events_England['playerId']! = 0]


In [122]:
eT, event_df = "Corner", Events_England

In [123]:
df1 = pd.DataFrame(event_df[event_df["subEventName"]== eT ][['matchId', "teamId", "subEventName"]])


In [117]:
# Function to transform match events into meaningul player performance profiles for machine learning
def match_agg(eT, event_df):

    df1 = pd.DataFrame(event_df[event_df["subEventName"]== eT ][['matchId', "teamId", "subEventName"]])
    df1 = df1.groupby(['matchId','teamId']).size().to_frame(eT).reset_index()
    df2 = df1.groupby('matchId').agg(['unique']).reset_index()
    df3 = pd.DataFrame(None, columns = ['matchId', 'teamId', eT])
    df3['matchId'], df3['teamId'], df3[eT] = df2.iloc[:, 0], df2.iloc[:,1], df2.iloc[:,2]
    df4 = pd.concat([df3, pd.DataFrame(df3['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
    result = pd.merge(df4, Matches[['matchId', "homeTeamId", "awayTeamId"]], on='matchId', how='inner')
    result = pd.concat([result, pd.DataFrame(df3[eT].to_list(), columns=[eT+"1",eT+"2"])], axis=1, sort = False)
    result = result.fillna(0) # handle na values where teams had no corners in a match

    # fill in zeros to teams where there were no corners for a team
    result[eT] = result[eT].apply(lambda row: list(row)+[0] if len(row)==1 else row)

    # Correctly apply allocate home and away aggregate statistics to correct teams
    result['home'+eT] = (
        np.select(
            condlist=[result['team1'] == result['homeTeamId'], result['team2'] == result['homeTeamId']], 
            choicelist=[result[eT+"1"], result[eT+"2"]]))

    result['away'+eT] = (
        np.select(
            condlist=[result['team1'] == result['awayTeamId'], result['team2'] == result['awayTeamId']], 
            choicelist=[result[eT+"1"], result[eT+"2"]]))
    
    # Create a column to recognize when a goal is scored
    conditions = [
        (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Success"))),
        (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Failure") & (result['subEventName'] == "Shot"))),
        (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Failure") & (result['subEventName'] != "Shot")))
        ]

    # create a list of the values we want to assign for each condition
    values = [1, 1, -1]

    # create a new column and use np.select to assign values to it using our lists as arguments
    result['Goal_Value'] = np.select(conditions, values, default = 0)
    
    #result = result[result['playerId']!= 0]

    # # drop useless columns that have been replaced
    result.drop(['teamId', eT, "team1", "team2", eT+"1", eT+"2"], inplace = True, axis = 1 )
    
    return result

In [ ]:
# # generate comprehensive dataframe with summary stat for a particular team-match attribute
# a = match_agg("Corner", Events_England) 
# b = match_agg("Corner", Events_France)
# c = match_agg("Corner", Events_Italy)
# d = match_agg("Corner", Events_Spain)
# e = match_agg("Corner", Events_Germany) 
# #f = match_agg("Corner", Events_World_Cup) 
# #g = match_agg("Corner", Events_EuroChamps) 

# match_aggs = pd.concat([a,b,c,d,e], ignore_index=True)


In [ ]:
### Successful Vertical Yards Now Merge

df = Events_England.loc[Events_England.Result == "Success" ].groupby(['matchId','playerId'])['attackMetres'].sum().reset_index()
df2 = Events_England.loc[Events_England.Result == "Failure" ].groupby(['matchId','playerId'])['attackMetres'].sum().reset_index()

# with names, positions and clubs for reference
#df = pd.merge(Events_England.loc[Events_England.Result == "Success" ].groupby(['matchId','playerId'])['attackYards'].sum().reset_index(), Players[['playerId', 'shortName', "Position", 'clubName']], on = 'playerId').sort_values(by ='attackYards', ascending = False)
#df2 = pd.merge(Events_England.loc[Events_England.Result == "Failure" ].groupby(['matchId','playerId'])['attackYards'].sum().reset_index(), Players[['playerId', 'shortName', "Position", 'clubName']], on = 'playerId').sort_values(by ='attackYards', ascending = False)

# .sort_values(by ='attackYards', ascending = False)



In [ ]:
advanced_stats(df, "attackMetres", Events_England)

In [ ]:
# dataframes of brute sum of attacking yard attempts without success or failure measures

#df  = pd.merge(Events_England.groupby(['matchId','playerId'])['attackYards'].agg(['sum']).reset_index(), Players[['playerId', 'shortName', "Position", 'clubName']], on ='playerId').sort_values(by ='sum', ascending = False).rename(columns={'sum':'attackYards'})
# df = Events_England.groupby(['matchId','playerId'])['attackYards'].agg(['sum']).reset_index().sort_values(by ='sum', ascending = False).rename(columns={'sum':'attackYards'})


### Data Processing

In [ ]:
#Match_TeamStats = pd.read_sql_query("SELECT * FROM football_data", sql_conn)

In [ ]:
# # parsing betting odds variables from analysis
# drop_columns = ["B365H","B365D","B365A","BSH","BSD","BSA","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA",
# "PSH", "PH","PSD", "PD","PSA", "PA","SOH","SOD","SOA","SBH","SBD","SBA","SJH","SJD","SJA","SYH","SYD","SYA","VCH",
# "VCD","VCA","WHH","WHD","WHA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","MaxH","MaxD","MaxA",
# "AvgH","AvgD","AvgA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","GB>2.5","GB<2.5","B365>2.5","B365<2.5",
# "P>2.5","P<2.5","Max>2.5","Max<2.5","Avg>2.5","Avg<2.5","BbAH","BbAHh","AHh","BbMxAHH","BbAvAHH","BbMxAHA",
# "BbAvAHA","GBAHH","GBAHA","GBAH","LBAHH","LBAHA","LBAH","B365AHH","B365AHA","B365AH","PAHH", "AvgC<2.5",
# "PAHA","MaxAHH","MaxAHA","AvgAHH","AvgAHA","WHCH","WHCD","WHCA", "VCCH", "VCCA", "PSCH", "PSCD", "PCAHH", "PCAHA", "PC>2.5", "PC<2.5", "MaxCH", "MaxCAHH", "MaxCD",
# "MaxCAHA", "MaxCA", "MaxC>2.5", "MaxC<2.5", "IWCH", "IWCD", "IWCA", "AvgC>2.5","AvgCA","AvgCAHA","AvgCAHH",
# "AvgCD","AvgCH","B365C<2.5","B365C>2.5","B365CA","B365CAHA","B365CAHH","B365CD","B365CH","BWCA", "BWCD", "BWCH", "VCCD", "PSCA"] 

In [ ]:
# Match_TeamStats.drop(drop_columns, inplace=True, axis =1) # removing betting odds from dataframe

In [ ]:
def df_analysis(df, df_name):
    header = "Dimension of " + str(df_name) + " Dataframe is: {}"
    print(header.format(df.shape))
    print(100*"*")
    print(df.info())
    print(100*"*")
    print(df.select_dtypes(exclude=['float64','int64']).describe())
    print(100*"*")
    print(df.describe())
    print(100*"*")
    print("Number of Null Values")
    print(df.isnull().sum(axis=0))

In [ ]:
# df_analysis(Match_TeamStats, "Match_TeamStats")

In [ ]:
# match[match["HomeTeam"].str.contains("Man United")]

### Outlier Detection and Understanding